
PROBLEMA - Canivales y misioneros

ESTADOS (c,m) en el lado inicial

ACEPTADOS - (3,3),(3,0), (2,2), (2,3), (2,0), (1,0), (1,1), (1,3), (0,3), (0,0)
NO ACEPTADOS - (3,1), (3,2), (2,1), (1,2), (0,1), (0,2)

ACCIONES sumar o restar c y m alternadamente al estado actual
{(0,1),(0,2),(1,0),(1,1),(2,0)}

FUNCION DE TRANSICION
f : SxA -> S

INICIO (3,3)
META (0,0)


In [101]:
class Estado:

    def __init__(self,c,m,i=0):
        self.c = c
        self.m = m
        self.i = i

    def setm(self, m):
        self.m = m

    def setc(self, c):
        self.c = c
    
    def increm(self):
        self.i += 1

    def getm(self):
        return self.m

    def getc(self):
        return self.c
    
    def geti(self):
        return self.i

    def isValid(self):
        return (self.c,self.m) not in [(3,1), (3,2), (2,1), (1,2), (0,1), (0,2)]

    def __repr__(self):
        return 'Canivales: {} Misioneros: {}'.format(self.c,self.m)
                                                      
    def __eq__(self, other):
        return self.c == other.c and self.m == other.m
    
    def __hash__(self):
        return hash((self.c,self.m,self.i))

In [102]:
class Problem(object):

    """The abstract class for a formal problem. You should subclass0
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

In [103]:
class Node:

    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state.  Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node.  Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node parent_action: {}, state: {},  depth: {}>".format(self.action, self.state, self.depth)

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        """[Figure 3.10]"""
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action,
                    problem.path_cost(self.path_cost, self.state,
                                      action, next_state))
        return next_node
    
    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    
    def getDepth(self):
        return self.depth

import math
failState = Estado(-1,-1)
failure = Node(failState, path_cost=math.inf) # Indicates an algorithm couldn't find a solution.

In [104]:
class Cym(Problem):
    # (c,m) (0,1),(0,2),(1,0),(1,1),(2,0)
    initial = Estado(3,3)
    goal = Estado(0,0)
    
    def __init__(self):
        Problem.__init__(self,self.initial,self.goal)

    def actions(self, estado):
        actions = []
        if estado.geti()%2 == 0: # es par, es decir, va a restar
            if estado.c >= 2:
                actions = actions + [(1,0),(2,0)]
                if estado.m >= 2:
                    actions = actions + [(0,1),(0,2),(1,1)]
                elif estado.m == 1:
                    actions = actions + [(0,1),(1,1)]
            elif estado.c == 1:
                actions.append((1,0))
                if estado.m >= 2:
                    actions = actions + [(0,1),(0,2),(1,1)]
                elif estado.m == 1:
                    actions = actions + [(0,1),(1,1)]
        else:
            if 3-estado.c >= 2:
                actions = actions + [(1,0),(2,0)]
                if 3-estado.m >= 2:
                    actions = actions + [(0,1),(0,2),(1,1)]
                elif 3-estado.m == 1:
                    actions = actions + [(0,1),(1,1)]
            elif 3-estado.c == 1:
                actions.append((1,0))
                if 3-estado.m >= 2:
                    actions = actions + [(0,1),(0,2),(1,1)]
                elif 3-estado.m == 1:
                    actions = actions + [(0,1),(1,1)]
        return actions
    
    def result(self, state, action):
        if state.geti() % 2 == 0: # es par, es decir, va a restar
            nuevo = Estado(state.getc()-action[0], state.getm()-action[1], state.geti()+1)
        else: # es impar, es decir, va a sumar
            nuevo = Estado(state.getc()+action[0], state.getm()+action[1], state.geti()+1)
        if nuevo.isValid():
            return nuevo
        else:
            return failState
        
    def goal_test(self, state):
        return state == self.goal

In [105]:
from collections import deque

FIFOQueue = deque

def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.goal_test(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in node.expand(problem):
            s = child.state
            if problem.goal_test(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure


In [106]:
problema = Cym()
print(problema.initial)
print(problema.goal)


Canivales: 3 Misioneros: 3
Canivales: 0 Misioneros: 0


In [107]:
bf = breadth_first_search(problema)

In [108]:
repr(bf.solution())

'[(2, 0), (1, 0), (2, 0), (1, 0), (0, 2), (1, 1), (0, 2), (1, 0), (2, 0), (1, 0), (2, 0)]'

In [109]:
node = Node(Estado(3,3))
for child in node.expand(problema):
    s = child.state
    print(type(s))
    print(s)

<class '__main__.Estado'>
Canivales: 2 Misioneros: 3
<class '__main__.Estado'>
Canivales: 1 Misioneros: 3
<class '__main__.Estado'>
Canivales: -1 Misioneros: -1
<class '__main__.Estado'>
Canivales: -1 Misioneros: -1
<class '__main__.Estado'>
Canivales: 2 Misioneros: 2


In [110]:
for a in bf.path():
    print(a)

<Node parent_action: None, state: Canivales: 3 Misioneros: 3,  depth: 0>
<Node parent_action: (2, 0), state: Canivales: 1 Misioneros: 3,  depth: 1>
<Node parent_action: (1, 0), state: Canivales: 2 Misioneros: 3,  depth: 2>
<Node parent_action: (2, 0), state: Canivales: 0 Misioneros: 3,  depth: 3>
<Node parent_action: (1, 0), state: Canivales: 1 Misioneros: 3,  depth: 4>
<Node parent_action: (0, 2), state: Canivales: 1 Misioneros: 1,  depth: 5>
<Node parent_action: (1, 1), state: Canivales: 2 Misioneros: 2,  depth: 6>
<Node parent_action: (0, 2), state: Canivales: 2 Misioneros: 0,  depth: 7>
<Node parent_action: (1, 0), state: Canivales: 3 Misioneros: 0,  depth: 8>
<Node parent_action: (2, 0), state: Canivales: 1 Misioneros: 0,  depth: 9>
<Node parent_action: (1, 0), state: Canivales: 2 Misioneros: 0,  depth: 10>
<Node parent_action: (2, 0), state: Canivales: 0 Misioneros: 0,  depth: 11>
